In [7]:
from faker import Faker
import random
import mysql.connector
from datetime import datetime, timedelta
import json
# Khởi tạo Faker
fake = Faker()

# Kết nối tới cơ sở dữ liệu
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Camtruykich123",  # Mật khẩu cho cơ sở dữ liệu
    database="ecommerce_optimized"
)
cursor = connection.cursor()

# Hàm sinh dữ liệu
def generate_customers(n):
    customers = []
    for _ in range(n):
        name = fake.name()[:255]
        email = fake.unique.email()[:255]
        phone = fake.phone_number().replace('(', '').replace(')', '').replace(' ', '').replace('-', '')[:20]
        address = fake.address().replace('\n', ', ')[:255]
        city = fake.city()[:100]
        country = fake.country()[:100]
        is_vip = random.choice([0, 1])
        loyalty_points = random.randint(0, 5000)
        created_at = fake.date_time_this_year()
        customers.append((name, email, phone, address, city, country, is_vip, loyalty_points, created_at, created_at))
    return customers

def generate_users(customers):
    users = []
    password = "Camtruykich123"  # Mật khẩu mặc định cho tất cả người dùng
    for customer_id in range(1, len(customers) + 1):
        username = fake.unique.user_name()[:255]
        email = customers[customer_id - 1][1]
        role = random.choice(['customer', 'admin']) if customer_id % 20 == 0 else 'customer'
        created_at = fake.date_time_this_year()
        users.append((username, password, email, role, customer_id, created_at, created_at))
    return users

def generate_categories(n):
    categories = []
    for _ in range(n):
        name = fake.unique.word().capitalize()[:255]
        description = fake.text(max_nb_chars=100)[:255]
        created_at = fake.date_time_this_year()
        categories.append((name, description, created_at, created_at))
    return categories

def generate_products(categories):
    products = []
    for category_id, category in enumerate(categories, start=1):
        for _ in range(random.randint(10, 50)):
            title = f"{fake.word().capitalize()} {fake.word().capitalize()}"[:255]
            description = fake.text(max_nb_chars=200)[:255]
            price = round(random.uniform(5, 1000), 2)
            images = json.dumps([fake.image_url() for _ in range(random.randint(1, 5))])
            created_at = fake.date_time_this_year()
            products.append((title, description, price, images, category_id, created_at, created_at, 0))
    return products

def generate_product_inventory(products):
    inventory = []
    for product_id, product in enumerate(products, start=1):
        for _ in range(random.randint(1, 5)):
            change_type = random.choice(['added', 'sold', 'returned'])
            quantity = random.randint(1, 100)
            created_at = fake.date_time_this_year()
            inventory.append((product_id, change_type, quantity, created_at))
    return inventory

def generate_orders(customers):
    orders = []
    for customer_id, customer in enumerate(customers, start=1):
        for _ in range(random.randint(1, 10)):
            total_price = round(random.uniform(50, 5000), 2)
            status = random.choice(['pending', 'paid', 'shipped', 'completed', 'cancelled'])
            created_at = fake.date_time_this_year()
            orders.append((customer_id, total_price, status, created_at, created_at, 0))
    return orders

def generate_order_items(orders, products):
    order_items = []
    for order_id, order in enumerate(orders, start=1):
        for _ in range(random.randint(1, 5)):
            product_id = random.randint(1, len(products))
            quantity = random.randint(1, 5)
            price = round(random.uniform(10, 100), 2)
            order_items.append((order_id, product_id, quantity, price))
    return order_items

def generate_vouchers(n):
    vouchers = []
    for _ in range(n):
        code = fake.unique.uuid4()[:8].upper()
        discount_value = round(random.uniform(5, 50), 2)
        required_points = random.randint(100, 5000)
        expires_at = fake.date_time_between(start_date="now", end_date="+1y")
        created_at = fake.date_time_this_year()
        vouchers.append((code, discount_value, required_points, expires_at, created_at, created_at))
    return vouchers

def generate_customer_vouchers(customers, vouchers):
    customer_vouchers = []
    for customer_id in range(1, len(customers) + 1):
        for _ in range(random.randint(1, 3)):
            voucher_id = random.randint(1, len(vouchers))
            status = random.choice(['unused', 'used', 'expired'])
            created_at = fake.date_time_this_year()
            customer_vouchers.append((customer_id, voucher_id, status, None, created_at, created_at))
    return customer_vouchers

def generate_cart_items(customers, products):
    cart_items = []
    for customer_id in range(1, len(customers) + 1):
        for _ in range(random.randint(0, 3)):
            product_id = random.randint(1, len(products))
            quantity = random.randint(1, 5)
            created_at = fake.date_time_this_year()
            updated_at = created_at + timedelta(days=random.randint(1, 30))
            cart_items.append((customer_id, product_id, quantity, created_at, updated_at))
    return cart_items

def generate_vip_history(customers):
    vip_history = []
    reasons = ['Top5_Monthly', 'Top3_Category', 'Spending_Over_Threshold']
    for customer_id in range(1, len(customers) + 1):
        if customers[customer_id - 1][6]:  # is_vip == 1
            promotion_reason = random.choice(reasons)
            created_at = fake.date_time_this_year()
            vip_history.append((customer_id, promotion_reason, created_at))
    return vip_history

# Chèn dữ liệu vào bảng
def insert_into_table(query, data):
    cursor.executemany(query, data)
    connection.commit()

# Hàm seed toàn bộ dữ liệu
def seed_database():
    print("Generating customers...")
    customers = generate_customers(500)
    insert_into_table("""
        INSERT INTO customers (name, email, phone, address, city, country, is_vip, loyalty_points, created_at, updated_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, customers)

    print("Generating users...")
    users = generate_users(customers)
    insert_into_table("""
        INSERT INTO users (username, password, email, role, customer_id, created_at, updated_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, users)

    print("Generating categories...")
    categories = generate_categories(20)
    insert_into_table("""
        INSERT INTO categories (name, description, created_at, updated_at)
        VALUES (%s, %s, %s, %s)
    """, categories)

    print("Generating products...")
    products = generate_products(categories)
    insert_into_table("""
        INSERT INTO products (title, description, price, images, category_id, created_at, updated_at, deleted)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, products)

    print("Generating product inventory...")
    inventory = generate_product_inventory(products)
    insert_into_table("""
        INSERT INTO product_inventory (product_id, change_type, quantity, created_at)
        VALUES (%s, %s, %s, %s)
    """, inventory)

    print("Generating orders...")
    orders = generate_orders(customers)
    insert_into_table("""
        INSERT INTO orders (customer_id, total_price, status, created_at, updated_at, deleted)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, orders)

    print("Generating order items...")
    order_items = generate_order_items(orders, products)
    insert_into_table("""
        INSERT INTO order_items (order_id, product_id, quantity, price)
        VALUES (%s, %s, %s, %s)
    """, order_items)

    print("Generating vouchers...")
    vouchers = generate_vouchers(300)
    insert_into_table("""
        INSERT INTO vouchers (code, discount_value, required_points, expires_at, created_at, updated_at)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, vouchers)

    print("Generating customer vouchers...")
    customer_vouchers = generate_customer_vouchers(customers, vouchers)
    insert_into_table("""
        INSERT INTO customer_vouchers (customer_id, voucher_id, status, order_id, created_at, updated_at)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, customer_vouchers)

    print("Generating cart items...")
    cart_items = generate_cart_items(customers, products)
    insert_into_table("""
        INSERT INTO cart_items (customer_id, product_id, quantity, created_at, updated_at)
        VALUES (%s, %s, %s, %s, %s)
    """, cart_items)

    print("Generating VIP history...")
    vip_history = generate_vip_history(customers)
    insert_into_table("""
        INSERT INTO vip_history (customer_id, promotion_reason, created_at)
        VALUES (%s, %s, %s)
    """, vip_history)

# Thực thi
try:
    seed_database()
    print("Database seeding complete!")
except Exception as e:
    print(f"Error occurred: {e}")
finally:
    cursor.close()
    connection.close()


Generating customers...
Generating users...
Generating categories...
Generating products...
Generating product inventory...
Generating orders...
Generating order items...
Generating vouchers...
Generating customer vouchers...
Generating cart items...
Generating VIP history...
Database seeding complete!
